In [0]:
from datetime import datetime, timedelta

In [0]:
dbutils.widgets.text('p_catalog_name','')
dbutils.widgets.text('p_schema_name','')
dbutils.widgets.text('p_table_name','')
dbutils.widgets.text('p_storage_account_baseURL','')
dbutils.widgets.text('p_container','')
dbutils.widgets.text('p_control_schema','')
dbutils.widgets.text('p_control_table','')
# dbutils.widgets.dropdown('p_load_type','incremental',['full','incremental'])

catalog_name = dbutils.widgets.get('p_catalog_name')
schema_name = dbutils.widgets.get('p_schema_name')
table_name = dbutils.widgets.get('p_table_name')
storage_account = dbutils.widgets.get('p_storage_account_baseURL')
container = dbutils.widgets.get('p_container')
control_schema = dbutils.widgets.get('p_control_schema')
control_table = dbutils.widgets.get('p_control_table')
# load_type = dbutils.widgets.get('p_load_type')

In [0]:
last_load_date = spark.sql(f"SELECT last_loaded_date FROM stocks.control_schema.control_table").collect()[0][0]

next_date = (datetime.strptime(last_load_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")

In [0]:
if not spark.catalog.tableExists(f"{catalog_name}.{schema_name}.{table_name}"):
    # First-time load → full
    spark.sql(
        f"""
        CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.{table_name} (
            value STRING,
            ingestion_date TIMESTAMP,
            file_path STRING
        )
        USING delta
        """
    )

    spark.sql(
        f"""
        COPY INTO {catalog_name}.{schema_name}.{table_name}
        FROM (
            SELECT
                value,
                current_timestamp() AS ingestion_date,
                _metadata.file_path AS file_path
            FROM 'abfss://{container}@{storage_account}/stocks/raw/full_load/'
        )
        FILEFORMAT = TEXT
        COPY_OPTIONS ('mergeSchema' = 'true')
        """
    )
else:
    # Incremental
    spark.sql(
        f"""
        COPY INTO {catalog_name}.{schema_name}.{table_name}
        FROM (
            SELECT
                value,
                current_timestamp() AS ingestion_date,
                _metadata.file_path AS file_path
            FROM 'abfss://{container}@{storage_account}/stocks/raw/incremental/{next_date}/'
        )
        FILEFORMAT = TEXT
        COPY_OPTIONS ('mergeSchema' = 'true')
        """
    )


In [0]:
spark.sql(f"DELETE FROM {catalog_name}.{control_schema}.{control_table}")
spark.sql(f"INSERT INTO {catalog_name}.{control_schema}.{control_table} VALUES ('{next_date}')")

In [0]:
%sql
select count(*)
from stocks.bronze_dev.stocks_dev